# 新版建图
* 使用邻接矩阵代替for循环来建立图
* 

In [1]:
import numpy as np
import torch
from tqdm import tqdm
from collections import Counter
import torch.nn.functional as F
import dgl
from maintrain.utils.utils import Cluster, neighber_idx
import copy
torch.set_printoptions(threshold=np.inf)

Using backend: pytorch


In [10]:
def L2_dist(x, y):
    """
    a fast approach to compute X, Y euclidean distace.
    Args:
        x: pytorch Variable, with shape [m, d]
        y: pytorch Variable, with shape [n, d]
    Returns:
        dist: pytorch Variable, with shape [m, n]
    """
    #y: pytorch Variable, with shape [n, d]
    m, n = x.size(0), y.size(0)
    # xx经过pow()方法对每单个数据进行二次方操作后，在axis=1 方向（横向，就是第一列向最后一列的方向）加和，此时xx的shape为(m, 1)，经过expand()方法，扩展n-1次，此时xx的shape为(m, n)
    xx = torch.pow(x, 2).sum(1, keepdim=True).expand(m, n)
    # yy会在最后进行转置的操作
    yy = torch.pow(y, 2).sum(1, keepdim=True).expand(n, m).t()
    dist = xx + yy
    # torch.addmm(beta=1, input, alpha=1, mat1, mat2, out=None)，这行表示的意思是dist - 2 * x * yT 
    dist.addmm_(x, y.t(), beta=1, alpha=-2)
    # clamp()函数可以限定dist内元素的最大最小范围，dist最后开方，得到样本之间的距离矩阵
    dist = dist.clamp(min=1e-12).sqrt()  # for numerical stability
    #归一化一下
    dist_min =dist.min()
    dist = dist - dist_min
    dist_max = dist.max()
    dist = dist / dist_max
    return dist


array([  0., 111.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.])

In [ ]:
class new_graph:
    def __init__(self) -> None:
        pass
    def init_adj_matric(self):
        '''
        建立固定的邻接矩阵

        '''
        return np.random.random(300,300)
    def init_edge(self):
        """初始化边，参考javed
        """
        e_ij = []
        e_pos = []
        #仿照javed公式写(r,c,h,w)分别代表了 左上角的坐标和图像的高和宽
        h = w = 128
        f_ij = L2_dist(self.node_fea, self.node_fea)#公式中||f_i - f_j||_2
        d_ij = L2_dist(self.d, self.d)#公式中d_ij
        #公式中p_ij
        #DONE 只算最近邻
        px =  self.d.permute(1, 0)[0] # 所有x坐标
        px1 = px.expand(px.size(0), px.size(0))
        px2 = px1.permute(1, 0)
        py = self.d.permute(1, 0)[1] # 所有的y坐标
        py1 = py.expand(py.size(0), py.size(0))
        py2 = py1.permute(1, 0) # 
        p_ij1 = ((px1 - px2) / h)
        p_ij2 = ((py1 - py2) / h)
        #这里需要每个分量都是N * N

        # 只有近邻有P值
        #reshape一下，方便操作 先空着
        p_ij1 = p_ij1.view(self.node_num, self.node_num)
        p_ij2 = p_ij2.view(self.node_num, self.node_num)
        #先建立{(x, y): idx}，用于根据位置检索邻居的idx
        pos_idx_dict = {}
        for i in range(len(self.wsi_dic)):
            # pos = self.wsi_dic[i][2]
            # pos_idx_dict[tuple(pos)] = i
            pos_x, pos_y = int(self.wsi_dic[i][2][0]), int(self.wsi_dic[i][2][1])
            pos_idx_dict[(pos_x, pos_y)] = i
        for i in range(len(p_ij1)):
            #只保留近邻的值
            #先计算邻居的idx
            pos = self.wsi_dic[i][2]
            nei_idx = neighber_idx(pos, 1, pos_idx_dict)
            temp = torch.tensor([k if idx in nei_idx else 0 for idx, k in enumerate(p_ij1[i])])
            p_ij1[i] = temp
            temp = torch.tensor([k if idx in nei_idx else 0 for idx, k in enumerate(p_ij2[i])])
            p_ij2[i] = temp

        self.node_num = len(self.node_fea)
        z = torch.zeros_like(f_ij).to(self.device)
        # print(f"各种大小{f_ij.size()} {p_ij1.size()} {p_ij2.size()} {d_ij.size()} {z.size()}")
        # edge_fea = torch.stack([f_ij, p_ij1, p_ij2, z, z, d_ij])
        edge_fea = torch.stack([f_ij])
        edge_pos = torch.stack([p_ij1, p_ij2, z, z, d_ij])
        edge_fea = edge_fea.permute(2, 1, 0) # 大小是3,3,3 其中edge_fea[i][j]就代表了那个点的特征
        edge_pos = edge_pos.permute(2, 1, 0)
        edge_pos = self.edge_mlp(edge_pos)
        edge_fea = edge_fea.contiguous().view(1, -1)
        edge_pos = edge_pos.contiguous().view(1, -1)
        #此时edge_fea 为 [n, n, edge_fea_dim]
        # for i in tqdm(range(self.node_num)): #全连接图
        #     for j in range(self.node_num):
        #         e_ij.append(edge_fea[i][j])
        #         e_pos.append(edge_pos[i][j])
        # e_ij = torch.stack(e_ij)
        # e_pos = torch.stack(e_pos)
        # print(f"test{e_ij.size()}")
        #e_ij是特征维度的矩阵， e_pos是物理维度的矩阵
        #[n^2, 6] --> [n, n]
        return edge_fea.view(self.node_num, self.node_num), edge_pos.view(self.node_num, self.node_num)
    
        
